In [1]:
import math
import json
import numpy as np
import matplotlib.pyplot as plt

import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe
import openrtdynamics2.targets as tg
from openrtdynamics2.ORTDtoNumpy import ORTDtoNumpy
from openrtdynamics2.manifest_import import show_inputs, show_outputs

from vehicle_lib import *
import vehicle_lib.path_transformations as pt
from vehicle_lib.motion_primitives import *

# Generate simulink s-functions

Using this notebook, a subset of the algorithms implemented in the library are exported to Simulink blocks. Therefore, they can be used in Simulink diagrams to build larger control systems or simulations. The generated code is written into the folder sfunctions/ .

In [2]:
code_gen_results, compiled_system = pt.compile_lateral_path_transformer(
    target_template = tg.TargetCppSimulinkSFunction(sfun_name='lateral_path_transformer'),
    folder          = 'sfunctions/'
)

compiling system Sys1000_store_input_data (level 1)... 
compiling system Sys1003_optim_loop (level 3)... 
compiling system Sys1002_tracker (level 2)... 
compiling system Sys1004_controller (level 2)... 
compiling system Sys1005_simulation_model (level 2)... 
compiling system Sys1001_process_data (level 1)... 
compiling system simulation (level 0)... 
Generated code will be written to sfunctions/ .
writing file sfunctions/simulation_manifest.json
writing file sfunctions/lateral_path_transformer.cpp
writing file sfunctions/lateral_path_transformer_block_mask_commands.m


### Show a summary of the blocks connectors.

In [3]:
show_inputs( code_gen_results['manifest'] )

+-------+------------------------+---------+--------+-------+----------------+--------------------------------------------------------------------+
| #port | input signal,  to -->  | outputs | update | reset | datatype (c++) | description                                                        |
+-------+------------------------+---------+--------+-------+----------------+--------------------------------------------------------------------+
|   0   |           Ts           |         |   X    |       |     double     | sampling time [s]                                                  |
|   1   |       wheelbase        |         |   X    |       |     double     | wheelbase (l_r) [m]                                                |
|   2   |       velocity_        |    X    |   X    |       |     double     | vehicle velocity [m/s]                                             |
|   3   |       Delta_l_r        |    X    |        |       |     double     | lateral deviation to the path [m]

In [4]:
show_outputs(code_gen_results['manifest'] )

+-------+--------------------------------+----------------+
| #port |          input signal          | datatype (c++) |
+-------+--------------------------------+----------------+
|   0   |          output_valid          |      bool      |
|   1   |   need_more_path_input_data    |      bool      |
|   2   | distance_at_the_end_of_horizon |     double     |
|   3   |         distance_ahead         |     double     |
|   4   |           head_index           |    int32_t     |
|   5   |         read_position          |    int32_t     |
|   6   |     elements_free_to_write     |    int32_t     |
|   7   |         tracked_index          |    int32_t     |
|   8   |          path_d_star           |     double     |
|   9   |             path_d             |     double     |
|   10  |             path_x             |     double     |
|   11  |             path_y             |     double     |
|   12  |            path_psi            |     double     |
|   13  |             path_K            

In [5]:
# show the manifest

from IPython.display import JSON, Code
JSON(code_gen_results['manifest'], expanded=False)

<IPython.core.display.JSON object>